<a href="https://colab.research.google.com/github/Salmaag/NLP/blob/main/models/AUTOCORRECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://cdn.discordapp.com/attachments/778630432878362676/849552709488607242/data_qalb.rar

In [ ]:
!unrar x /content/data_qalb.rar

In [1]:
!wget https://cdn.discordapp.com/attachments/778630432878362676/848807810690449408/t_1.txt
!wget https://cdn.discordapp.com/attachments/778630432878362676/848930769942478878/chars2_2.txt
!wget https://cdn.discordapp.com/attachments/778630432878362676/848955187430424676/wordChars_2_1.txt

--2021-06-07 16:43:38--  https://cdn.discordapp.com/attachments/778630432878362676/848807810690449408/t_1.txt
Resolving cdn.discordapp.com (cdn.discordapp.com)... 162.159.130.233, 162.159.135.233, 162.159.129.233, ...
Connecting to cdn.discordapp.com (cdn.discordapp.com)|162.159.130.233|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1028 (1.0K) [text/plain]
Saving to: ‘t_1.txt’

t_1.txt             100%[===================>]   1.00K  --.-KB/s    in 0s      

2021-06-07 16:43:39 (161 MB/s) - ‘t_1.txt’ saved [1028/1028]

--2021-06-07 16:43:39--  https://cdn.discordapp.com/attachments/778630432878362676/848930769942478878/chars2_2.txt
Resolving cdn.discordapp.com (cdn.discordapp.com)... 162.159.129.233, 162.159.130.233, 162.159.135.233, ...
Connecting to cdn.discordapp.com (cdn.discordapp.com)|162.159.129.233|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 95 [text/plain]
Saving to: ‘chars2_2.txt’

chars2_2.txt        100%[=============

In [2]:
import codecs
import unicodedata
from itertools import groupby
import numpy as np
from __future__ import division
from __future__ import print_function
import re
import copy

In [3]:
class Node:
    "class representing nodes in a prefix tree"

    def __init__(self):
        self.children = {}  # all child elements beginning with current prefix
        self.isWord = False  # does this prefix represent a word

    def __str__(self):
        s = ''
        for k in self.children.keys():
            s += k
        return 'isWord: ' + str(self.isWord) + '; children: ' + s


class PrefixTree:
    "prefix tree"

    def __init__(self):
        self.root = Node()

    def addWord(self, text):
        "add word to prefix tree"
        node = self.root
        for i in range(len(text)):
            c = text[i]  # current char
            if c not in node.children:
                node.children[c] = Node()
            node = node.children[c]
            isLast = (i + 1 == len(text))
            if isLast:
                node.isWord = True

    def addWords(self, words):
        for w in words:
            self.addWord(w)

    def getNode(self, text):
        "get node representing given text"
        node = self.root
        for c in text:
            if c in node.children:
                node = node.children[c]
            else:
                return None
        return node

    def isWord(self, text):
        node = self.getNode(text)
        if node:
            return node.isWord
        return False

    def getNextChars(self, text):
        "get all characters which may directly follow given text"
        chars = []
        node = self.getNode(text)
        if node:
            for k in node.children.keys():
                chars.append(k)
        return chars

    def getNextWords(self, text):
        "get all words of which given text is a prefix (including the text itself, it is a word)"
        words = []
        node = self.getNode(text)
        if node:
            nodes = [node]
            prefixes = [text]
            while len(nodes) > 0:
                # put all children into list
                for k, v in nodes[0].children.items():
                    nodes.append(v)
                    prefixes.append(prefixes[0] + k)

                # is current node a word
                if nodes[0].isWord:
                    words.append(prefixes[0])

                # remove current node
                del nodes[0]
                del prefixes[0]

        return words

    def dump(self):
        nodes = [self.root]
        while len(nodes) > 0:
            # put all children into list
            for v in nodes[0].children.values():
                nodes.append(v)

            # dump current node
            print(nodes[0])

            # remove from list
            del nodes[0]

In [4]:
class Optical:
    "optical score of beam"

    def __init__(self, prBlank=0, prNonBlank=0):
        self.prBlank = prBlank  # prob of ending with a blank
        self.prNonBlank = prNonBlank  # prob of ending with a non-blank


class Textual:
    "textual score of beam"

    def __init__(self, text=''):
        self.text = text
        self.wordHist = []  # history of words so far
        self.wordDev = ''  # developing word
        self.prUnnormalized = 1.0
        self.prTotal = 1.0


class Beam:
    "beam with text, optical and textual score"

    def __init__(self, lm, useNGrams):
        "creates genesis beam"
        self.optical = Optical(1.0, 0.0)
        self.textual = Textual('')
        self.lm = lm
        self.useNGrams = useNGrams

    def mergeBeam(self, beam):
        "merge probabilities of two beams with same text"

        if self.getText() != beam.getText():
            raise Exception('mergeBeam: texts differ')

        self.optical.prNonBlank += beam.getPrNonBlank()
        self.optical.prBlank += beam.getPrBlank()

    def getText(self):
        return self.textual.text

    def getPrBlank(self):
        return self.optical.prBlank

    def getPrNonBlank(self):
        return self.optical.prNonBlank

    def getPrTotal(self):
        return self.getPrBlank() + self.getPrNonBlank()

    def getPrTextual(self):
        return self.textual.prTotal

    def getNextChars(self):
        return self.lm.getNextChars(self.textual.wordDev)

    def createChildBeam(self, newChar, prBlank, prNonBlank):
        "extend beam by new character and set optical score"
        beam = Beam(self.lm, self.useNGrams)

        # copy textual information
        beam.textual = copy.deepcopy(self.textual)
        beam.textual.text += newChar

        # do textual calculations only if beam gets extended
        if newChar != '':
            if self.useNGrams:  # use unigrams and bigrams

                # if new char occurs inside a word
                if newChar in beam.lm.getWordChars():
                    beam.textual.wordDev += newChar
                    nextWords = beam.lm.getNextWords(beam.textual.wordDev)

                    # no complete word in text, then use unigram of all possible next words
                    numWords = len(beam.textual.wordHist)
                    prSum = 0
                    if numWords == 0:
                        for w in nextWords:
                            prSum += beam.lm.getUnigramProb(w)
                    # take last complete word and sum up bigrams of all possible next words
                    else:
                        lastWord = beam.textual.wordHist[-1]
                        for w in nextWords:
                            prSum += beam.lm.getBigramProb(lastWord, w)
                    beam.textual.prTotal = beam.textual.prUnnormalized * prSum
                    beam.textual.prTotal = beam.textual.prTotal ** (
                            1 / (numWords + 1)) if numWords >= 1 else beam.textual.prTotal

                # if new char does not occur inside a word
                else:
                    # if current word is not empty, add it to history
                    if beam.textual.wordDev != '':
                        beam.textual.wordHist.append(beam.textual.wordDev)
                        beam.textual.wordDev = ''

                        # score with unigram (first word) or bigram (all other words) probability
                        numWords = len(beam.textual.wordHist)
                        if numWords == 1:
                            beam.textual.prUnnormalized *= beam.lm.getUnigramProb(beam.textual.wordHist[-1])
                            beam.textual.prTotal = beam.textual.prUnnormalized
                        elif numWords >= 2:
                            beam.textual.prUnnormalized *= beam.lm.getBigramProb(beam.textual.wordHist[-2],
                                                                                 beam.textual.wordHist[-1])
                            beam.textual.prTotal = beam.textual.prUnnormalized ** (1 / numWords)

            else:  # don't use unigrams and bigrams, just keep wordDev up to date
                if newChar in beam.lm.getWordChars():
                    beam.textual.wordDev += newChar
                else:
                    beam.textual.wordDev = ''

        # set optical information
        beam.optical.prBlank = prBlank
        beam.optical.prNonBlank = prNonBlank
        return beam

    def __str__(self):
        return '"' + self.getText() + '"' + ';' + str(self.getPrTotal()) + ';' + str(self.getPrTextual()) + ';' + str(
            self.textual.prUnnormalized)


class BeamList:
    "list of beams at specific time-step"

    def __init__(self):
        self.beams = {}

    def addBeam(self, beam):
        "add or merge new beam into list"
        # add if text not yet known
        if beam.getText() not in self.beams:
            self.beams[beam.getText()] = beam
        # otherwise merge with existing beam
        else:
            self.beams[beam.getText()].mergeBeam(beam)

    def getBestBeams(self, num):
        "return best beams, specify the max. number of beams to be returned (beam width)"
        u = [v for (_, v) in self.beams.items()]
        lmWeight = 1
        return sorted(u, reverse=True, key=lambda x: x.getPrTotal() * (x.getPrTextual() ** lmWeight))[:num]

    def deletePartialBeams(self, lm):
        "delete beams for which last word is not finished"
        for (k, v) in self.beams.items():
            lastWord = v.textual.wordDev
            if (lastWord != '') and (not lm.isWord(lastWord)):
                del self.beams[k]

    def completeBeams(self, lm):
        "complete beams such that last word is complete word"
        for (_, v) in self.beams.items():
            lastPrefix = v.textual.wordDev
            if lastPrefix == '' or lm.isWord(lastPrefix):
                continue

            # get word candidates for this prefix
            words = lm.getNextWords(lastPrefix)
            # if there is just one candidate, then the last prefix can be extended to
            if len(words) == 1:
                word = words[0]
                v.textual.text += word[len(lastPrefix) - len(word):]

    def dump(self):
        for k in self.beams.keys():
            print(unicode(self.beams[k]).encode('ascii', 'replace'))  # map to ascii if possible (for py2 and windows)


In [5]:
class LanguageModel:
    "unigram/bigram LM, add-k smoothing"

    def __init__(self, corpus, chars, wordChars):
        "read text from filename, specify chars which are contained in dataset, specify chars which form words"
        # read from file
        self.wordCharPattern = '[' + wordChars + ']'
        self.wordPattern = self.wordCharPattern + '+'
        words = re.findall(self.wordPattern, corpus)
        uniqueWords =  list(set(words)) 
        self.numWords = len(words)
        self.numUniqueWords = len(uniqueWords)
        self.smoothing = True
        self.addK = 1.0 if self.smoothing else 0.0
        #print(words)

        # create unigrams
        self.unigrams = {}
        for w in words:
            w = w.lower()
            if w not in self.unigrams:
                self.unigrams[w] = 0
            self.unigrams[w] += 1 / self.numWords

        # create unnormalized bigrams
        bigrams = {}
        for i in range(len(words) - 1):
            w1 = words[i].lower()
            w2 = words[i + 1].lower()
            if w1 not in bigrams:
                bigrams[w1] = {}
            if w2 not in bigrams[w1]:
                bigrams[w1][w2] = self.addK  # add-K
            bigrams[w1][w2] += 1

        # normalize bigrams
        for w1 in bigrams.keys():
            # sum up
            probSum = self.numUniqueWords * self.addK  # add-K smoothing
            for w2 in bigrams[w1].keys():
                probSum += bigrams[w1][w2]
            # and divide
            for w2 in bigrams[w1].keys():
                bigrams[w1][w2] /= probSum
        self.bigrams = bigrams

        # create prefix tree
        self.tree = PrefixTree()  # create empty tree
        self.tree.addWords(uniqueWords)  # add all unique words to tree

        # list of all chars, word chars and nonword chars
        self.allChars = chars
        self.wordChars = wordChars
        self.nonWordChars = str().join(
            set(chars) - set(re.findall(self.wordCharPattern, chars)))  # else calculate those chars

    def getNextWords(self, text):
        "text must be prefix of a word"
        return self.tree.getNextWords(text)

    def getNextChars(self, text):
        "text must be prefix of a word"
        nextChars = str().join(self.tree.getNextChars(text))

        # if in between two words or if word ends, add non-word chars
        if (text == '') or (self.isWord(text)):
            #print(text,'م',nextChars)
            nextChars += self.getNonWordChars()

        return nextChars

    def getWordChars(self):
        return self.wordChars

    def getNonWordChars(self):
        return self.nonWordChars

    def getAllChars(self):
        return self.allChars

    def isWord(self, text):
        return self.tree.isWord(text)

    def getUnigramProb(self, w):
        "prob of seeing word w."
        w = w.lower()
        val = self.unigrams.get(w)
        if val != None:
            return val
        return 0

    def getBigramProb(self, w1, w2):
        "prob of seeing words w1 w2 next to each other."
        w1 = w1.lower()
        w2 = w2.lower()
        val1 = self.bigrams.get(w1)
        if val1 != None:
            val2 = val1.get(w2)
            if val2 != None:
                return val2
            return self.addK / (self.getUnigramProb(w1) * self.numUniqueWords + self.numUniqueWords)
        return 0


In [6]:
def wordBeamSearch(mat, beamWidth, lm, useNGrams):
    "decode matrix, use given beam width and language model"
    chars = lm.getAllChars()
    blankIdx = len(chars)  # blank label is supposed to be last label in RNN output
    maxT, _ = mat.shape  # shape of RNN output: TxC


    genesisBeam = Beam(lm, useNGrams)  # empty string
    last = BeamList()  # list of beams at time-step before beginning of RNN output
    last.addBeam(genesisBeam)  # start with genesis beam

    # go over all time-steps
    for t in range(maxT):
        curr = BeamList()  # list of beams at current time-step

        # go over best beams
        bestBeams = last.getBestBeams(beamWidth)  # get best beams
        for beam in bestBeams:
            # calc probability that beam ends with non-blank
            prNonBlank = 0
            if beam.getText() != '':
                # char at time-step t must also occur at t-1
                labelIdx = chars.index(beam.getText()[-1])
                prNonBlank = beam.getPrNonBlank() * mat[t, labelIdx]

            # calc probability that beam ends with blank
            prBlank = beam.getPrTotal() * mat[t, blankIdx]

            # save result
            curr.addBeam(beam.createChildBeam('', prBlank, prNonBlank))

            # extend current beam with characters according to language model
            nextChars = beam.getNextChars()
            for c in nextChars:
                # extend current beam with new character
                #print(c)
                labelIdx = chars.index(c)
                if beam.getText() != '' and beam.getText()[-1] == c:
                    prNonBlank = mat[t, labelIdx] * beam.getPrBlank()  # same chars must be separated by blank
                else:
                    prNonBlank = mat[t, labelIdx] * beam.getPrTotal()  # different chars can be neighbours

                # save result
                curr.addBeam(beam.createChildBeam(c, 0, prNonBlank))

        # move current beams to next time-step
        last = curr

    # return most probable beam
    last.completeBeams(lm)
    bestBeams = last.getBestBeams(1)  # sort by probability
    return bestBeams[0].getText()

In [7]:
class Tokenizer():
    def __init__(self, chars, max_text_length=333):
        self.chars =  list(chars)
        self.vocab_size = len(self.chars)
        self.maxlen = max_text_length

    def encode(self, text):
        text = unicodedata.normalize("NFKD", text).encode("utf-8", "ignore").decode("utf-8")
        text = " ".join(text.split())


        #groups = ["".join(group) for _, group in groupby(text)]
        groups = [group for group in text]
        #print(groups)
        text = "".join([self.UNK_TK.join(list(x)) if len(x) > 1 else x for x in groups])
        encoded = []

        text = list(text)
        for item in text:
          index = self.chars.index(item)
          index = self.UNK if index == -1 else index
          encoded.append(index)
        return np.asarray(encoded)

    def decode(self, text):        
        decoded = "".join([self.chars[int(x)] for x in text if x > -1])
        return decoded

    def remove_tokens(self, text):
        return text.replace(self.PAD_TK, "").replace(self.UNK_TK, "")

In [8]:
chars = codecs.open('/content/chars2_2.txt', 'r', 'utf8').read()
wordChars = codecs.open('/content/wordChars_2_1.txt', 'r', 'utf8').read()
lm = LanguageModel(codecs.open('/content/t_1.txt').read(), chars, wordChars)

In [9]:
tokenizer = Tokenizer(chars)

In [10]:
def correct(sentence):
  split = sentence.split()
  sent = []
  for token in split:
    text = tokenizer.encode(token)
    text_to_correct = []
    for i in range(len(text)):
      letter_list = [0.0 for i in range(len(tokenizer.chars)+1)]
      letter_list[text[i]] = 1.0
      letter_list[-1] = -9999
      text_to_correct.append(letter_list)
    text_to_correct = np.array(text_to_correct)
    cor = wordBeamSearch(text_to_correct, 5, lm,True) 
    sent.append(cor)
  return  " ".join(sent)

In [13]:
def wer(r, h):
    import numpy
    size = len(r)
    d = numpy.zeros((len(r) + 1) * (len(h) + 1), dtype=numpy.uint8)
    d = d.reshape((len(r) + 1, len(h) + 1))
    for i in range(len(r) + 1):
        for j in range(len(h) + 1):
            if i == 0:
                d[0][j] = j
            elif j == 0:
                d[i][0] = i

    for i in range(1, len(r) + 1):
        for j in range(1, len(h) + 1):
            if r[i - 1] == h[j - 1]:
                d[i][j] = d[i - 1][j - 1]
            else:
                substitution = d[i - 1][j - 1] + 1
                insertion = d[i][j - 1] + 1
                deletion = d[i - 1][j] + 1
                d[i][j] = min(substitution, insertion, deletion)

    return d[len(r)][len(h)] , d[len(r)][len(h)]/size

In [157]:
t_1 = "الى التعليق رقم 2 اكيد ان لحكام العرب والمسلمين مسؤولية يتمثل ادناها في استدعاء السفراء في الصين للتشاور فليتهم يفعلونها ولو مرة ولنا نحن كشعوب مسؤولية كذالك تتمثل في مسانده اخواننا في الصين بمقاطعة البضائع الصينينة وليتنا نفعلها ولو ثلاتة اشهر"
t_2 = "كذالك في مسانده الصين بمقاطعة البضائع الصينينة"
t_3 = "نعم احرقها بطائراته التي تقصف المدن والمناطق السكنية احرقها بالبراميل التي ترمى من طائرات روسيا احرقها بمدافعه الثقيلة وذخيرته الايرانية احرق حمص لكي يبقى الطريق الى حزب الله بيده ولكي يؤمن ممر نجاة لدولته العلوية التي يسعى لها في حال سقوط مستعمرته سورية"
t_4= 'وهي موجهة على الاغلب ضد الشيعة و في مناطق مكتظة بالشسيعة والسنة'

correct_sent = [
           'الى التعليق رقم 2 اكيد ان للحكام العرب والمسلمين مسؤولية يتمثل ادناها في استدعاء السفراء في الصين للتشاور فليتهم يفعلونها ولو مرة ولنا نحن كشعوب مسؤولية كذلك تتمثل في مساندة اخواننا في الصين بمقاطعة البضائع الصينية وليتنا نفعلها ولو ثلاتة اشهر'
          , 'كذلك في مساندة الصين بمقاطعة البضائع الصينية'
           ,'نعم احرقها بطائراته التي تقصف المدن والمناطق السكنية احرقها بالبراميل التي ترمى من طائرات روسيا احرقها بمدافعه الثقيلة وذخيرته الايرانية احرق حمص لكي يبقى الطريق الى حزب الله بيده ولكي يؤمن ممر نجاة لدولته العلوية التي يسعى لها في حال سقوط مستعمرته سوريا',
           'وهي موجهة على الاغلب ضد الشيعة و في مناطق مكتظة بالشيعة والسنة']

In [ ]:
t_2_co = correct(t_2)
print(' original text = ',correct_sent[1],'\n','Wrong text = ',t_2,'\n','corrected text = ',t_2_co)
print(' WER = ',wer(correct_sent[1].split(),t_2_co.split())[1])

 original text =  كذلك في مساندة الصين بمقاطعة البضائع الصينية 
 Wrong text =  كذالك في مسانده الصين بمقاطعة البضائع الصينينة 
 corrected text =  كذلك في مساندة الصين بمقاطعة البضائع الصينية
 WER =  0.0


In [ ]:
t_1_co = correct(t_1)
print(' original text = ',correct_sent[0],'\n','Wrong text = ',t_1,'\n','corrected text = ',t_1_co)
print(' WER = ',wer(correct_sent[0].split(),t_1_co.split())[1])

 original text =  الى التعليق رقم 2 اكيد ان للحكام العرب والمسلمين مسؤولية يتمثل ادناها في استدعاء السفراء في الصين للتشاور فليتهم يفعلونها ولو مرة ولنا نحن كشعوب مسؤولية كذلك تتمثل في مساندة اخواننا في الصين بمقاطعة البضائع الصينية وليتنا نفعلها ولو ثلاتة اشهر 
 Wrong text =  الى التعليق رقم 2 اكيد ان لحكام العرب والمسلمين مسؤولية يتمثل ادناها في استدعاء السفراء في الصين للتشاور فليتهم يفعلونها ولو مرة ولنا نحن كشعوب مسؤولية كذالك تتمثل في مسانده اخواننا في الصين بمقاطعة البضائع الصينينة وليتنا نفعلها ولو ثلاتة اشهر 
 corrected text =  الى التعليق رقم 2 اكيد ان لها العرب والمسلمين مستعمرته يتمثل ادناها في استدعاء السفراء في الصين لها فليتهم يفعلونها ولو مرة ولنا نحن كشعوب مستعمرته كذلك تتمثل في مساندة اخواننا في الصين بمقاطعة البضائع الصينية وليتنا نفعلها ولو ثلاتة اشهر
 WER =  0.0975609756097561


In [ ]:
t_3_co = correct(t_3)
print(' original text = ',correct_sent[2],'\n','Wrong text = ',t_3,'\n','corrected text = ',t_3_co)
print(' WER = ',wer(correct_sent[2].split(),t_3_co.split())[1])

 original text =  نعم احرقها بطائراته التي تقصف المدن والمناطق السكنية احرقها بالبراميل التي ترمى من طائرات روسيا احرقها بمدافعه الثقيلة وذخيرته الايرانية احرق حمص لكي يبقى الطريق الى حزب الله بيده ولكي يؤمن ممر نجاة لدولته العلوية التي يسعى لها في حال سقوط مستعمرته سوريا 
 Wrong text =  نعم احرقها بطائراته التي تقصف المدن والمناطق السكنية احرقها بالبراميل التي ترمى من طائرات روسيا احرقها بمدافعه الثقيلة وذخيرته الايرانية احرق حمص لكي يبقى الطريق الى حزب الله بيده ولكي يؤمن ممر نجاة لدولته العلوية التي يسعى لها في حال سقوط مستعمرته سورية 
 corrected text =  نعم احرقها بطائراته التي تقصف المدن والمناطق السكنية احرقها بالبراميل التي ترمى من طائرات روسيا احرقها بمدافعه الثقيلة وذخيرته الايرانية احرقها حمص لكي يبقى الطريق الى حزب الا بيده ولكي يبقى من نجاة لدولته العلوية التي يسعى لها في حال سقوط مستعمرته سوريا
 WER =  0.09302325581395349


In [ ]:
t_4_co = correct(t_4)
print(' original text = ',correct_sent[3],'\n','Wrong text = ',t_4,'\n','corrected text = ',t_4_co)
print(' WER = ',wer(correct_sent[3].split(),t_4_co.split())[1])

 original text =  وهي موجهة على الاغلب ضد الشيعة و في مناطق مكتظة بالشيعة والسنة 
 Wrong text =  وهي موجهة على الاغلب ضد الشيعة و في مناطق مكتظة بالشسيعة والسنة 
 corrected text =  وهي موجهة على الاغلب ضد الشيعة و في مناطق مكتظة بالشيعة والسنة
 WER =  0.0


In [6]:
total_wer()

total error rate: 0.10667
